In [1]:
import os
import logging
from datetime import datetime, timedelta
import pandas as pd
import pandas as pd_f

import matplotlib.pyplot as plt

import numpy as np
import requests
import requests_cache
from retry_requests import retry
import plotly.graph_objects as go

import torch
from autogluon.timeseries import TimeSeriesPredictor, TimeSeriesDataFrame
import openmeteo_requests

import plotly.offline as pyo

import plotly.io as pio



# ------------------------------------------------
# Setup Logging and Plotly Configurations
# ------------------------------------------------
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
#pio.renderers.default = "browser"  # Opens Plotly figures in your browser

# ------------------------------------------------
# Setup Requests Caching and Retry Mechanism
# ------------------------------------------------
cache_session = requests_cache.CachedSession('.cache', expire_after=3600)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

# ------------------------------------------------
# Check for Apple MPS (GPU) Availability (Using MPS if available)
# ------------------------------------------------
if torch.backends.mps.is_available():
    logger.info("MPS (Apple GPU) is available. Using MPS for GPU acceleration.")
    device = "mps"
else:
    logger.info("MPS not available. Using CPU.")
    device = "cpu"

# ------------------------------------------------
# Function to Fetch Production Data
# ------------------------------------------------
def fetch_production_data(start_date: str, forecast_date: str) -> pd.DataFrame:
    url = (f'http://209.38.208.230:8000/api/pvmeasurementdata/'
           f'?start_date={start_date}&end_date={forecast_date}&ppe=590310600030911897')
    try:
        response = requests.get(url=url)
        response.raise_for_status()
        data = response.json()
        logger.info("Successfully fetched history production data.")
        return pd.DataFrame(data)
    except requests.exceptions.RequestException as e:
        logger.error(f"Error fetching production data: {e}")
        return pd.DataFrame()
   #
   # 
def fetch_production_data_future(start_date: str, forecast_date: str) -> pd_f.DataFrame:
    url = (f'http://209.38.208.230:8000/api/pvmeasurementdata/'
           f'?start_date={start_date}&end_date={forecast_date}&ppe=590310600030911897')
    try:
        response = requests.get(url=url)
        response.raise_for_status()
        data = response.json()
        logger.info("Successfully fetched actual production data.")
        return pd_f.DataFrame(data)
    except requests.exceptions.RequestException as e:
        logger.error(f"Error fetching production data: {e}")
        return pd_f.DataFrame()

    #
# ------------------------------------------------
# Function to Clean Production Data
# ------------------------------------------------
def clean_production_data(df: pd.DataFrame) -> pd.DataFrame:
    if df.empty:
        logger.warning("Production data is empty.")
        return df
    df['production'] = df['production'].replace(['-', 'n/e', 'N/A', 'NaN'], np.nan).astype(float)
    df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce', utc=True)
    df['timestamp'] = df['timestamp'].dt.tz_convert('Europe/Warsaw').dt.tz_localize(None)
    if df.isnull().values.any():
        logger.warning("NaNs detected in production data. Consider handling them appropriately.")
    df = df[['timestamp', 'production', 'latitude', 'longitude']].dropna()
    return df

# ------------------------------------------------
# Function to Fetch Weather Data with Error Handling
# ------------------------------------------------
def fetch_weather_data(start: str, end: str, 
                       url_weather: str = "https://archive-api.open-meteo.com/v1/archive") -> pd.DataFrame:
    try:
        lat = float(df_dam['latitude'].iloc[0])
        lon = float(df_dam['longitude'].iloc[0])
    except Exception as e:
        logger.error(f"Error retrieving latitude and longitude: {e}")
        return pd.DataFrame()
    
    params = {
        "latitude": lat,
        "longitude": lon,
        "start_date": start,    
        "end_date": end,
        "hourly": ["temperature_2m", "cloud_cover", "cloud_cover_low",
                   "wind_speed_10m", "direct_radiation", "diffuse_radiation",
                   "global_tilted_irradiance"],
        "tilt": 30
        # azimuth: 180 (south)
    }
    try:
        responses = openmeteo.weather_api(url_weather, params=params)
        response_weather = responses[0]
        hourly = response_weather.Hourly()
        # Extract weather variables
        hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
        hourly_cloud_cover = hourly.Variables(1).ValuesAsNumpy()
        hourly_cloud_cover_low = hourly.Variables(2).ValuesAsNumpy()
        hourly_wind_speed_10m = hourly.Variables(3).ValuesAsNumpy()
        hourly_direct_radiation = hourly.Variables(4).ValuesAsNumpy()
        hourly_diffuse_radiation = hourly.Variables(5).ValuesAsNumpy()
        hourly_global_tilted_irradiance = hourly.Variables(6).ValuesAsNumpy()
        
        # Create a date range based on API time info
        date_range = pd.date_range(
            start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
            end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
            freq=pd.Timedelta(seconds=hourly.Interval()),
            inclusive="left"
        )
        
        hourly_data = {
            "date": date_range,
            "temperature_2m": hourly_temperature_2m,
            "cloud_cover": hourly_cloud_cover,
            "cloud_cover_low": hourly_cloud_cover_low,
            "wind_speed_10m": hourly_wind_speed_10m,
            "direct_radiation": hourly_direct_radiation,
            "diffuse_radiation": hourly_diffuse_radiation,
            "global_tilted_irradiance": hourly_global_tilted_irradiance
        }
        hourly_df = pd.DataFrame(hourly_data)
        hourly_df["date"] = pd.to_datetime(hourly_df["date"])
        hourly_df.set_index("date", inplace=True)
        # Use "15min" to avoid FutureWarning
        resampled_df = hourly_df.resample("15min").ffill().reset_index()
        resampled_df["date"] = resampled_df["date"].dt.tz_localize(None)
        logger.info("Successfully fetched weather data.")
        return resampled_df
    except Exception as e:
        logger.error(f"Error fetching weather data: {e}")
        return pd.DataFrame()

# ------------------------------------------------
# Main Pipeline
# ------------------------------------------------
if __name__ == '__main__':
   
    # Define start and end dates for production data
    start_date = '2025-01-01'
    forecast_date = '2025-02-01'
    future_date = '2025-02-05'

    forecast_horizont = 3
    history_horizont = -3 # for the visualizations
    
    
    # Fetch and clean production data
    df_raw = fetch_production_data(start_date,forecast_date)
    df_dam = clean_production_data(df_raw)
    df_future = fetch_production_data(forecast_date, datetime.strptime(forecast_date, '%Y-%m-%d') + timedelta(days=forecast_horizont))

    if df_dam.empty:
        logger.error("Production data is empty. Exiting.")
        raise SystemExit("No production data available.")
    
    
    # ------------------------------------------------
    # Visualization 1: Production Heatmap (Plotly)
    # ------------------------------------------------
    try:
        df_vis = df_dam.copy()
        df_vis['WeekNumber'] = df_vis['timestamp'].dt.isocalendar().week
        df_weeks = df_vis[(df_vis['WeekNumber'] >= 1) & (df_vis['WeekNumber'] <= 52)]
        df_weeks['Date'] = df_weeks['timestamp'].dt.date
        df_weeks['Hour'] = df_weeks['timestamp'].dt.hour
        
        aggregated_data = df_weeks.groupby(['Hour', 'Date', 'WeekNumber'])['production'].mean().reset_index()
        heatmap_data = aggregated_data.pivot(index='Hour', columns='Date', values='production')
        
        fig_heat = go.Figure(data=go.Heatmap(
            z=heatmap_data.values,
            x=heatmap_data.columns.astype(str),
            y=heatmap_data.index,
            colorscale='Viridis',
            colorbar=dict(title="Production")
        ))
        fig_heat.update_layout(
            title='DAM Market Production Heatmap',
            xaxis_title='Date',
            yaxis_title='Hour of Day',
            yaxis_nticks=24,
            height=600
        )
        fig_heat.show()
    except Exception as e:
        logger.error(f"Error during production heatmap visualization: {e}")
    
    # ---------------------------------------------------------------------
    # Visualization 2: Hourly Production Over Time (Plotly)
    # ---------------------------------------------------------------------
    try:
        # Create an interactive scatter line plot for production over time
        fig_ts = go.Figure()
        fig_ts.add_trace(go.Scatter(
            x=df_weeks['timestamp'],
            y=df_weeks['production'],
            mode='lines+markers',
            name='Production'
        ))
        fig_ts.update_layout(
            title="Hourly Production Over Time",
            xaxis_title="Date and Hour",
            yaxis_title="Production",
            template="plotly_white"
        )
        fig_ts.show()
    except Exception as e:
        logger.error(f"Error during time series visualization: {e}")
    
    # ------------------------------------------------
    # Fetch Weather Data and Merge with Production Data
    # ------------------------------------------------
    resampled_df = fetch_weather_data(start_date, forecast_date)
    if resampled_df.empty:
        logger.error("Weather data is empty. Exiting.")
        raise SystemExit("No weather data available.")
    
    combined_df = pd.merge(df_dam, resampled_df, how='inner',
                           left_on='timestamp', right_on='date')
    combined_df.drop(columns=['date', 'latitude', 'longitude'], inplace=True)
    combined_df.dropna(inplace=True)
    combined_df = combined_df.iloc[:-1]  # Remove last row if needed
    logger.info("Combined production and weather data:")
    #logger.info(combined_df.tail())
    
    # ------------------------------------------------
    # Prepare Future Covariates (Forecast Data)
    # ------------------------------------------------
   
    start_date_val = forecast_date
    end_date_val = (datetime.strptime(forecast_date, '%Y-%m-%d') + timedelta(days=3)).strftime('%Y-%m-%d')
    forecast_df = fetch_weather_data(start_date_val, end_date_val, 
                                     url_weather="https://api.open-meteo.com/v1/forecast")
    if forecast_df.empty:
        logger.error("Forecast weather data is empty. Exiting.")
        raise SystemExit("No forecast weather data available.")
    
    forecast_df.rename(columns={'date': 'timestamp'}, inplace=True)
    forecast_df["item_id"] = "series_1"
    forecast_df = forecast_df.iloc[:288]  # Limit to prediction length
    
    if len(forecast_df) < 288:
        raise ValueError("Insufficient future covariates. Extend forecast horizon.")

    future_covariates = TimeSeriesDataFrame.from_data_frame(
        forecast_df.iloc[:288],  # Limit to prediction length
        id_column="item_id",
        timestamp_column="timestamp"
        )
    
    known_covariates = ["temperature_2m", "cloud_cover", "cloud_cover_low",
                        "wind_speed_10m", "direct_radiation", "diffuse_radiation",
                        "global_tilted_irradiance"]
    logger.info("Future covariates prepared:")
   # logger.info(future_covariates)
    
    # ------------------------------------------------
    # Visualization: Forecasted Weather Trends
    # ------------------------------------------------
    try:
        fig_forecast = go.Figure()
        
        # Add key weather variables to the plot
        fig_forecast.add_trace(go.Scatter(
            x=forecast_df["timestamp"],
            y=forecast_df["temperature_2m"],
            mode="lines",
            name="future temperature (°C)"
        ))
        fig_forecast.add_trace(go.Scatter(
            x=forecast_df["timestamp"],
            y=forecast_df["cloud_cover"],
            mode="lines",
            name="future Cloud Cover (%)"
        ))
        fig_forecast.add_trace(go.Scatter(
            x=forecast_df["timestamp"],
            y=forecast_df["global_tilted_irradiance"],
            mode="lines",
            name="future global_tilted_irradiance (W/m²)"
        ))
        fig_forecast.add_trace(go.Scatter(
            x=df_future["timestamp"],
            y=df_future["production"],
            mode="lines",
            name="future production"
        ))

        # Update layout for readability
        fig_forecast.update_layout(
            title="Forecasted Weather Trends",
            xaxis_title="Timestamp",
            yaxis_title="Value",
            legend_title="Weather Variables",
            template="plotly_dark"
        )

        fig_forecast.show()
    except Exception as e:
        logger.error(f"Error during forecast visualization: {e}")

    # ------------------------------------------------
    # Prepare Training Data for AutoGluon
    # ------------------------------------------------
    combined_df["item_id"] = "series_1"
    target_column = 'production'
    
    train_data = TimeSeriesDataFrame.from_data_frame(
        combined_df,
        id_column="item_id",
        timestamp_column="timestamp"
    )
    
    # ------------------------------------------------
    # Define Full Hyperparameters Including DeepAR with Verbose Progress
    # ------------------------------------------------
    full_hyperparameters = { 
        "DeepAR": {
            "epochs": 2,
            "learning_rate": 1e-3,
            "batch_size": 32,
            "verbose": 1,
            "early_stop_patience": 5
        },
    }
    predictor = TimeSeriesPredictor(
        target=target_column,    
        prediction_length=288,# да се провери дължината и да се нагалси с прямао другите данни
        freq='15min',
        known_covariates_names=known_covariates,
        path="AutogluonModels/ag_custom"
    )
    
    try:
        results = predictor.fit(
            train_data=train_data,    
            time_limit=3000,           
            hyperparameters=full_hyperparameters,
            enable_ensemble=False,
            verbosity=3
        )
        
    except Exception as e:
        logger.error(f"Error during model training: {e}")
        raise SystemExit("Model training failed.")
    
    logger.info("Training completed.")
    logger.info("Leaderboard:")
    #logger.info(predictor.leaderboard())
    
    try:
        cv_results = predictor.fit_summary()
        logger.info("Fit summary:")
        logger.info(cv_results)
    except Exception as e:
        logger.error(f"Error retrieving fit summary: {e}")
    
    try:
        evaluation = predictor.evaluate(train_data)
        logger.info(f"Evaluation score: {evaluation}")
    except Exception as e:
        logger.error(f"Error during evaluation: {e}")
    
    model_info = predictor.info()
    logger.info(f"Model info: {model_info}")
    
    # ------------------------------------------------
    # Inspect the Best Model (Updated API)
    # ------------------------------------------------
    best_model_name = predictor.model_best
    logger.info(f"The best model is: {best_model_name}")
    
    try:
        best_model_info = predictor.get_model_info(best_model_name)
        logger.info("Detailed best model info:")
        logger.info(best_model_info)
    except Exception as e:
        logger.warning("Detailed best model info not available: " + str(e))
    
    # Save the predictor
    # predictor.save()
    # logger.info("Predictor saved successfully.")
       
    # ------------------------------------------------
    # Visualization 4: Actual vs. Predicted Production (Plotly)
    # ------------------------------------------------
    # try:
    #     predictions_best = predictor.predict(data=train_data, known_covariates=future_covariates)
    #     predictions_best = predictions_best.clip(lower=0)
    #     df_pred_best = predictions_best.reset_index()
        
    #     # Assume df_validation has actual production values with 'timestamp' and 'production'
    #     if 'df_validation' in globals() and not df_validation.empty:
    #         df_val = df_validation.sort_values('timestamp')
    #     else:
    #         df_val = pd.DataFrame()  # If not available, this trace will be skipped
    #     fig_actual = go.Figure()
    #     fig_actual.add_trace(go.Scatter(
    #         x=df_pred_best['timestamp'],
    #         y=df_pred_best['mean'], #??????????
    #         mode='lines+markers',
    #         name='Predicted'
    #     ))
        
    #     fig_actual.add_trace(go.Scatter(
    #         x=df_future['timestamp'],
    #         y=df_future['production'],
    #         mode='lines+markers',
    #         name='Actual',     
    #     ))
       
    #     fig_actual.show()
    # except Exception as e:
    #     logger.error("Error visualizing actual vs. predicted production: " + str(e))


    # ------------------------------------------------
    # Visualize Model Feature Importance (if supported)
    # ------------------------------------------------
    try:
        # Compute feature importance using the training data.
        # Some models may not support this; if not, an exception will be raised.
        fi = predictor.feature_importance(data=train_data)
        logger.info("Feature importance:")
        logger.info(fi)
        
        if not fi.empty:
            # Sort the feature importances for better visualization.
            fi_sorted = fi.sort_values(by='importance', ascending=True)
            plt.figure(figsize=(10, 6))
            plt.barh(fi_sorted.index, fi_sorted['importance'], color='skyblue')
            plt.xlabel("Importance")
            plt.title("Feature Importance of the Best Model")
            plt.tight_layout()
            plt.show()
        else:
            logger.warning("Feature importance DataFrame is empty.")
    except Exception as e:
        logger.warning("Feature importance not available or error computing it: " + str(e))

c:\Users\Georgi\test_gluon\gluon_venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:__main__:MPS not available. Using CPU.
INFO:__main__:Successfully fetched history production data.
INFO:__main__:Successfully fetched history production data.


INFO:__main__:Successfully fetched weather data.
INFO:__main__:Combined production and weather data:
INFO:__main__:Successfully fetched weather data.
INFO:__main__:Future covariates prepared:


Beginning AutoGluon training... Time limit = 3000s
AutoGluon will save models to 'c:\Users\Georgi\test_gluon\AutogluonModels\ag_custom'
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.9.10
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
CPU Count:          8
GPU Count:          0
Memory Avail:       2.74 GB / 15.75 GB (17.4%)
Disk Space Avail:   82.78 GB / 237.84 GB (34.8%)

Fitting with arguments:
{'enable_ensemble': False,
 'eval_metric': WQL,
 'freq': '15min',
 'hyperparameters': {'DeepAR': {'batch_size': 32,
                                'early_stop_patience': 5,
                                'epochs': 2,
                                'learning_rate': 0.001,
                                'verbose': 1}},
 'known_covariates_names': ['temperature_2m',
                            'cloud_cover',
                            'cloud_cover_low',
                            'wind_speed_10m',
      

KeyboardInterrupt: 

In [15]:
import os
import plotly.offline as pyo

# Initialize Plotly offline mode
pyo.init_notebook_mode(connected=True)

# Define the notebook name (without file extension)
notebook_name = 'forecast_improvement_toolbox'  # Change this to your notebook's actual name

# Convert Jupyter Notebook to HTML
convert_html_cmd = f'jupyter nbconvert --to html "{notebook_name}.ipynb"'
html_status = os.system(convert_html_cmd)

if html_status == 0:  # Check if the HTML conversion was successful
    print(f'Successfully converted {notebook_name}.ipynb to HTML.')
else:
    print(f'Error: Failed to convert {notebook_name}.ipynb to HTML. Ensure Jupyter is installed.')

Error: Failed to convert forecast_improvement_toolbox.ipynb to HTML. Ensure Jupyter is installed.


In [9]:
try:
    start_date = '2025-01-01'
    forecast_date = '2025-02-01'
    future_date = '2025-02-05'

    forecast_horizont = 3
    history_horizont = -3 # for the visualizations

    # forecast_date to datetime
    forecast_date = datetime.strptime(forecast_date, '%Y-%m-%d')
    # future_date to datetime
    future_date = datetime.strptime(future_date, '%Y-%m-%d')


    predictions_best = predictor.predict(data=train_data, known_covariates=future_covariates)
    predictions_best = predictions_best.clip(lower=0)
    df_pred_best = predictions_best.reset_index()

    ref_actual_production = fetch_production_data(forecast_date.strftime('%Y-%m-%d'), future_date.strftime('%Y-%m-%d'))
  
    # fetch weather forecast
    ref_weather_forecast = fetch_weather_data(forecast_date.strftime('%Y-%m-%d'), future_date.strftime('%Y-%m-%d'))
    ref_weather_forecast.rename(columns={'date': 'timestamp'}, inplace=True)

    
    fig_actual = go.Figure()

    # Prediction:
    fig_actual.add_trace(go.Scatter(
        x=df_pred_best['timestamp'],
        y=df_pred_best['0.9'], #?????????? Мисля, че трябва да е 0.9
        mode='lines+markers',
        name='Predicted'
    ))
    # Actual Production:
    fig_actual.add_trace(go.Scatter(
        x=ref_actual_production['timestamp'],
        y=ref_actual_production['production'],
        mode='lines+markers',
        name='Production Actual',     
    ))
    # Actual Weather top 3 features:
    fig_actual.add_trace(go.Scatter(
        x=ref_actual_production['timestamp'],
        y=ref_actual_production['global_tilted_irradiance'],
        mode='lines+markers',
        name='Tilted Irradiance Actual',     
    ))
    # fig_actual.add_trace(go.Scatter(
    #     x=ref_actual_production['timestamp'],
    #     y=ref_actual_production['diffuse_radiation'],
    #     mode='lines+markers',
    #     name='diffuse radiation',     
    # ))
    # Weather Forecast top 3 features
    fig_actual.add_trace(go.Scatter(
        x=ref_weather_forecast['timestamp'],
        y=ref_weather_forecast['diffuse_radiation'],
        mode='lines+markers',
        name='Diffuse Radiation forecast',     
    ))
    fig_actual.add_trace(go.Scatter(
        x=ref_weather_forecast['timestamp'],
        y=ref_weather_forecast['direct_radiation'],
        mode='lines+markers',
        name='direct radiation forecast',     
    ))
    fig_actual.add_trace(go.Scatter(
        x=ref_weather_forecast['timestamp'],
        y=ref_weather_forecast['global_tilted_irradiance'],
        mode='lines+markers',
        name='global tilted irradiance forecast',     
    ))
    

    
    fig_actual.show()
except Exception as e:
    logger.error("Error visualizing actual vs. predicted production: " + str(e))


Model not specified in predict, will default to the model with the best validation score: DeepAR
INFO:__main__:Successfully fetched history production data.
INFO:__main__:Successfully fetched weather data.


In [ ]:
# Find Similarities between a given day from the forecast weather and similar days from the historical data 
from sklearn.preprocessing import StandardScaler

def find_similar_days_gti(historical_data, forecast_data, top_n=3):
    column_to_compare = 'global_tilted_irradiance'
    
    # Extract the relevant data
    historical_gti = historical_data[column_to_compare].values
    forecast_gti = forecast_data[column_to_compare].values
    
    # Normalize the data
    scaler = StandardScaler()
    historical_normalized = scaler.fit_transform(historical_gti.reshape(-1, 1)).flatten()
    forecast_normalized = scaler.transform(forecast_gti.reshape(-1, 1)).flatten()
    
    # Ensure forecast data has 96 points
    if len(forecast_normalized) != 96:
        raise ValueError("Forecast data should have 96 points for a full day")
    
    # Reshape historical data into daily chunks, dropping incomplete days
    days_in_historical = len(historical_normalized) // 96
    historical_days = historical_normalized[:days_in_historical*96].reshape(days_in_historical, 96)
    
    # Calculate Euclidean distance between forecast and each historical day
    distances = np.sqrt(((historical_days - forecast_normalized) ** 2).sum(axis=1))
    
    # Find indices of the most similar days
    similar_days_indices = distances.argsort()[:top_n]
    
    # Get the dates of the most similar days
    similar_days_dates = historical_data.index[similar_days_indices * 96]
    
    return similar_days_dates, distances[similar_days_indices]


forecast_date_end = forecast_date + timedelta(days=1)
reference_weather_day = fetch_weather_data(forecast_date.strftime('%Y-%m-%d'), forecast_date_end.strftime('%Y-%m-%d'))
reference_weather_day.rename(columns={'date': 'timestamp'}, inplace=True)
reference_weather_day = reference_weather_day[:96]

historical_data = fetch_production_data(start_date, forecast_date)
cleared_historical_data = clean_production_data(historical_data)

similar_dates, similarities = find_similar_days_gti(historical_data, reference_weather_day)
print("Most similar days based on global tilted irradiance:", similar_dates)
print("Similarity scores:", similarities)





INFO:__main__:Successfully fetched weather data.


INFO:__main__:Successfully fetched history production data.


Most similar days based on global tilted irradiance: Index([2112, 1920, 1248], dtype='int64')
Similarity scores: [2.61601247 3.1443408  3.40021696]
               timestamp  production latitude longitude
1212 2025-01-13 15:00:00        84.6  52.6395   16.6405


In [ ]:
import pandas as pd
from openpyxl import load_workbook

# Load your DataFrame (assuming you already have it)
# forecast_data = pd.read_csv('your_data.csv')  # Uncomment if you need to load the data

# Load the existing Excel workbook
workbook = load_workbook('ZUSE_template.xlsx')

# Select the active sheet
sheet = workbook.active

# Get the data from the 'production_forecast' column
data_to_write = forecast_data['production_forecast'].tolist()

# Write the data horizontally starting from row 11, column 4
for col, value in enumerate(data_to_write, start=4):
    sheet.cell(row=11, column=col, value=value)

# Save the workbook
workbook.save('ZUSE_template_updated.xlsx')
